# fashinMNIST_Contest

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

import pandas as pd

In [4]:
def append_ext(fn):
    return fn+".png"

In [5]:
traindf=pd.read_csv("train.csv",dtype=str)
testdf=pd.read_csv("test.csv",dtype=str)

In [6]:
traindf["id"]=traindf["id"].apply(append_ext)
testdf["id"]=testdf["id"].apply(append_ext)

In [7]:
traindf

,id,label
0,1.png,9
1,2.png,0
2,3.png,0
3,4.png,3
4,5.png,0
...,...,...
59995,59996.png,5
59996,59997.png,1
59997,59998.png,3
59998,59999.png,0


In [8]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.1)

In [9]:
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="train",
x_col="id",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
color_mode="grayscale",
class_mode="categorical",
target_size=(28,28))

Found 54000 validated image filenames belonging to 10 classes.


In [10]:
valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="train",
x_col="id",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
color_mode="grayscale",
class_mode="categorical",
target_size=(28,28))

Found 6000 validated image filenames belonging to 10 classes.


In [11]:
test_datagen=ImageDataGenerator(rescale=1./255.)

In [12]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="test",
x_col="id",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
color_mode="grayscale",
class_mode=None,
target_size=(28,28))

Found 10000 validated image filenames.


In [13]:
# model2 = keras.models.Sequential([
#     keras.layers.Conv2D(filters=64, kernel_size=7,activation="relu",padding="same", input_shape=[28, 28, 1]),
#     keras.layers.MaxPooling2D(pool_size=2),
    
#     keras.layers.Conv2D(filters=128, kernel_size=3,activation="relu",padding="same"),
#     keras.layers.Conv2D(filters=128, kernel_size=3,activation="relu",padding="same"),
#     keras.layers.MaxPooling2D(pool_size=2),
    
#     keras.layers.Conv2D(filters=256, kernel_size=3,activation="relu",padding="same"),
#     keras.layers.Conv2D(filters=256, kernel_size=3,activation="relu",padding="same"),
#     keras.layers.MaxPooling2D(pool_size=2),
    
#     keras.layers.Flatten(),
#     keras.layers.Dense(units=128, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(units=64, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(units=10, activation='softmax'),
# ])

# model2.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [14]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu",input_shape=[28, 28, 1]),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

In [15]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TRAIN

1687

In [16]:
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_VALID

187

In [17]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
STEP_SIZE_TEST

312

In [19]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/3
1687/1687 [==============================] - 1438s 853ms/step - loss: 0.3935 - accuracy: 0.8608 - val_loss: 0.2675 - val_accuracy: 0.9036
Epoch 2/3
1687/1687 [==============================] - 136s 80ms/step - loss: 0.2884 - accuracy: 0.8973 - val_loss: 0.2264 - val_accuracy: 0.9126
Epoch 3/3
1687/1687 [==============================] - 134s 80ms/step - loss: 0.2457 - accuracy: 0.9107 - val_loss: 0.2255 - val_accuracy: 0.9208


In [ ]:
model.save("fashionMNIST_model.h5")

In [29]:
model = keras.models.load_model("fashionMNIST_model.h5")

In [30]:
model.evaluate_generator(generator=valid_generator,steps=STEP_SIZE_TEST)

[0.20647627115249634, 0.9316666722297668]

In [31]:
# test_generator.reset()
pred=model.predict_generator(test_generator, verbose=1)

313/313 [==============================] - 104s 332ms/step


In [32]:
predicted_class_indices=np.argmax(pred,axis=1)

In [33]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [34]:
submission = pd.read_csv('sample_submission_I5njJSF.csv')

submission['label'] = predictions

submission.to_csv('my_submission.csv', index=False)